<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2024/aoc2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Day 01, input

In [43]:
input = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input01.txt')]

Day 01, part 1, sum of absolute diff of sorted list elements

In [45]:
sum(abs(l - r) for (l, r) in zip(sorted([l for (l, _) in input]), sorted([r for (_, r) in input])))

1530215

Day 01, part 2, sum lhs elements multiplied by frequency in rhs list

In [46]:
from collections import Counter
r_counts = Counter(r for (_, r) in input)
sum(l * r_counts[l] for (l, _) in input)

26800609

Day 02, input

In [47]:
input = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input02.txt')]

Day 02, part 1, part 1, determine (all increasing or all decreasing) and deltas < 4

In [48]:
deltas = [[line[n+1] - line[n] for n in range(len(line)-1)] for line in input]

In [49]:
def same_sgn(deltas_line):
  return all(deltas_line[i] * deltas_line[i+1] > 0 for i in range(len(deltas_line)-1))

In [50]:
def delta_max(deltas_line, m):
  return all(abs(d) <= m for d in deltas_line)

In [51]:
sum(1 if same_sgn(deltas_line) and delta_max(deltas_line, 3) else 0 for deltas_line in deltas)

359

Day 02, part 2, allow removal of any single element

In [52]:
def check_line(line):
  deltas = [line[n+1] - line[n] for n in range(len(line)-1)]
  return same_sgn(deltas) and delta_max(deltas, 3)

def remove1(line):
  return [line[:i] + line[i+1:] for i in range(len(line))]

# part 1 regression
sum(1 if check_line(line) else 0 for line in input)

# part 2
sum(1 if check_line(line) or any(check_line(r) for r in remove1(line)) else 0 for line in input)

418

Day 03, input

In [53]:
input = "".join(line for line in open('drive/MyDrive/AoC/2024/input03.txt'))

Day 03, part 1, find mul(x,y) sequences, multiply and add

In [54]:
import re

def find_mul_and_eval(instructions):
  # regex mul\\((\\d+),(\\d+)\\)
  matches = re.findall("mul\((\d+),(\d+)\)", instructions)
  return sum(int(x) * int(y) for (x, y) in matches)

In [55]:
# input = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
find_mul_and_eval(input)

162813399

Day 03, part 2, evaluate do() and don't() sequences

In [56]:
import re
from functools import reduce

def next_state(state, split):
  (sum, is_active) = state
  match split:
    case "do()":
      return (sum, True)
    case "don't()":
      return (sum, False)
    case _:
      return (sum + (find_mul_and_eval(split) if is_active else 0), is_active)

# input = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
# split on (do\\(\\)|don't\\(\\)) - capturing group leads to delimiters being included in result
reduce(next_state, re.split("(do\(\)|don't\(\))", input), (0, True))[0]

53783319

Day 04, input

In [72]:
board = [line.strip() for line in open('drive/MyDrive/AoC/2024/input04.txt')]
rows = len(board)
cols = len(board[0])

Day 04, part 1, find XMAS

In [58]:
from functools import reduce

def in_range(p):
  r, c = p
  return r >= 0 and r < rows and c >= 0 and c < cols

def check_char(p, char):
  r, c = p
  return in_range(p) and board[r][c] == char

def check_xmas(p, d):
  def next_state(state, char):
    p, found = state
    r, c = p
    dr, dc = d
    return (r + dr, c + dc), found and check_char(p, char)

  p, found = reduce(next_state, "XMAS", (p, True))
  return found

deltas = [(dr, dc) for dr in [-1, 0, 1] for dc in [-1, 0, 1] if dr != 0 or dc !=0]

def check_xmas_all(p):
  return sum(check_xmas(p,d) for d in deltas)

sum(check_xmas_all((r, c)) for r in range(rows) for c in range(cols))


18

Day 05, part 2, find
```
M.S
.A.
M.S
```



In [77]:
# The pattern can only occur in 4 different variants, with 'A' at (0,0)
patterns = [
    {(-1,-1): 'M', (-1,1): 'S', (1,-1): 'M', (1,1): 'S'},
    {(-1,-1): 'M', (-1,1): 'M', (1,-1): 'S', (1,1): 'S'},
    {(-1,-1): 'S', (-1,1): 'M', (1,-1): 'S', (1,1): 'M'},
    {(-1,-1): 'S', (-1,1): 'S', (1,-1): 'M', (1,1): 'M'},
]

def vec_add(v1, v2):
  return tuple(sum(i) for i in zip(v1, v2))

def check_pattern(p, pattern):
  return all([check_char(vec_add(p, d), char) for (d, char) in pattern.items()])

# Search for A within (1,1)...(rows-1, cols-1) and check match for all patterns

sum(1
    for p in [(r, c) for r in range(1, rows-1) for c in range(1, cols-1) if board[r][c] == 'A']
    for pattern in patterns if check_pattern(p, pattern))

2029